In [52]:
import os
from tqdm import tqdm
import subprocess
import json
import pandas as pd

In [4]:
!pwd

/home/fenics/shared/mini_project_celery


In [5]:
!ls ../murtazo/cloudnaca/msh/

r0a0n200.msh   r0a9n200.msh   r1a6n200.msh   r2a27n200.msh  r3a21n200.msh
r0a12n200.msh  r1a0n200.msh   r1a9n200.msh   r2a27n200.xml  r3a24n200.msh
r0a15n200.msh  r1a12n200.msh  r2a0n200.msh   r2a30n200.msh  r3a27n200.msh
r0a18n200.msh  r1a15n200.msh  r2a12n200.msh  r2a3n200.msh   r3a30n200.msh
r0a21n200.msh  r1a18n200.msh  r2a12n200.xml  r2a6n200.msh   r3a30n200.xml
r0a24n200.msh  r1a21n200.msh  r2a15n200.msh  r2a9n200.msh   r3a3n200.msh
r0a24n200.xml  r1a21n200.xml  r2a18n200.msh  r3a0n200.msh   r3a6n200.msh
r0a27n200.msh  r1a24n200.msh  r2a18n200.xml  r3a12n200.msh  r3a9n200.msh
r0a30n200.msh  r1a27n200.msh  r2a21n200.msh  r3a15n200.msh
r0a3n200.msh   r1a30n200.msh  r2a24n200.msh  r3a18n200.msh
r0a6n200.msh   r1a3n200.msh   r2a24n200.xml  r3a18n200.xml


In [6]:
# --------------------- Local TESTING -------------------------------------------------------

In [7]:
def startSimulation(filename):
    #get path for filename and corresponding .xml file
    xmlfilename = input_path + filename.split(".")[0]+".xml"
    filename_path = input_path + filename
    
    #Task 1 convert msh to xml by dolfin-convert <msh> <xml>
    !dolfin-convert $filename_path $xmlfilename
#     subprocess.run(["dolfin-convert", filename_path, xmlfilename],check=True, capture_output=True)
    #Task 2 run navier_stokes_solver by ./airfoil 10 0.0001 10. 1 file
    ! ../murtazo/navier_stokes_solver/airfoil 10 0.00001 10. 1 $xmlfilename
#     subprocess.run(["../murtazo/navier_stokes_solver/airfoil", 10,0.0001,10.,1, xmlfilename],check=True, capture_output=True)
    
    #Task3 getResults by Tail –n 1 drag_ligt.m
    output = !tail -n 1  ../murtazo/navier_stokes_solver/results/drag_ligt.m 
#     subprocess.run(["tail", "-n",1, "../murtazo/navier_stokes_solver/results/drag_ligt.m"],check=True, capture_output=True)
    #preprocessing result into dictionary
    result = {"filename": filename, "lift": output[0].split("\t")[1], "drag": output[0].split("\t")[2]}
    return output, result, json.dumps(result)

%%time
results =  []
input_path = "../murtazo/cloudnaca/msh/"
sublist_msh = ['r0a24n200.msh','r0a0n200.msh',   'r0a1n200.msh',  'r0a2n200.msh', 'r0a3n200.msh','r0a4n200.msh','r1a27n200.msh'  ]
# for filename in tqdm(os.listdir(input_path)):
for filename in tqdm(sublist_msh):
    output, result, json = startSimulation(filename)
    results.append(output)
    break
    
    

In [8]:
%%time
results =  []
input_path = "../murtazo/cloudnaca/msh/"
sublist_msh = ['r0a24n200.msh','r0a0n200.msh',   'r0a1n200.msh',  'r0a2n200.msh', 'r0a3n200.msh','r0a4n200.msh','r1a27n200.msh'  ]
# for filename in tqdm(os.listdir(input_path)):
for filename in tqdm(sublist_msh):
    output, result, json = startSimulation(filename)
    results.append(output)
    break
    
    
    
 

  0%|          | 0/7 [00:00<?, ?it/s]

/bin/bash: line 1: dolfin-convert: command not found
../murtazo/navier_stokes_solver/airfoil: error while loading shared libraries: libdolfin.so.2019.1: cannot open shared object file: No such file or directory


  0%|          | 0/7 [00:02<?, ?it/s]

CPU times: user 18 ms, sys: 7.57 ms, total: 25.6 ms
Wall time: 2.84 s


In [10]:
# print(output, type(output))
# print(result, type(result))
# print(json, type(json))


['1\t37.0841\t13.1751'] <class 'IPython.utils.text.SList'>
{'filename': 'r0a24n200.msh', 'lift': '37.0841', 'drag': '13.1751'} <class 'dict'>
{"filename": "r0a24n200.msh", "lift": "37.0841", "drag": "13.1751"} <class 'str'>


In [11]:
# # Task 3 run navier_stokes_solver by ./airfoil 10 0.0001 10. 1 file
# print(results)

[['1\t37.0841\t13.1751']]


In [6]:
#------------ CELERY IMPLEMENTATION -----------------------------------------------------


In [49]:
%%time
import os
from tqdm import tqdm
from proj.celery import startSimulation
taskqueue =  []
input_path = "../murtazo/cloudnaca/msh/"
# Scaling Test with only three mesh files
sublist_msh = ['r0a24n200.msh', 'r0a0n200.msh', 'r0a3n200.msh'  ]
# sublist_msh = ['r0a24n200.msh', 'r0a0n200.msh', 'r0a3n200.msh','r1a27n200.msh'   ]
i = 0
#for filename in tqdm(os.listdir(input_path)):
for filename in tqdm(sublist_msh):
    if "xml" in filename:
        print(filename)
    else:
        output = startSimulation.delay(filename, input_path)
        taskqueue.append(output)
        i = i+1
    
list_of_DicResults = []
for asyncResult in taskqueue:
    list_of_DicResults.append(asyncResult.get())
    
print(list_of_DicResults)

100%|██████████| 3/3 [00:00<00:00, 241.83it/s]


['{"filename": "r0a24n200.msh", "lift": "25.3015", "drag": "19.7578\\n"}', '{"filename": "r0a0n200.msh", "lift": "2.64065", "drag": "17.2411\\n"}', '{"filename": "r0a3n200.msh", "lift": "-0.702569", "drag": "17.8132\\n"}']
CPU times: user 63.4 ms, sys: 3.36 ms, total: 66.8 ms
Wall time: 1min 40s
